# Lecture 27 Debugging 

debugging - finding bugs and removing them

What are bugs?

Any unintenional, unwanted behaviour in your code
- Error that makes your code not run
- Code runs, but doesn't give the correct output
- Code runs with some inputs, but crashes with other inputs

Today we will cover

- Python error types
- Handling errors
- How to read an error traceback
- Methods to find the bug
- Methods to avoid bugs

### Error types

Syntax Error - When there is incorrect code that stops Python from being able to "interpret" the code

Code won't start running at all

In [ ]:
x = 100
print(x)

if x > 2
    print(x)

First print didn't happen, even though SyntaxError is after it. No code ran at all

In [ ]:
x = 100
print(x)

if x > 2:
print(x)

Same here. Some Syntax Errors have more specific names

Logical errors

In Python a.k.a. Exceptions 

These do not stop the code from executing at all, but will raise an "Exception" when the line containting this error runs

In [ ]:
x = 100
print(x)

if x > 2:
    print(x/0)
print(x)

The code started to run, until it got to the error in line 5

In [ ]:
x = 100
print(x)

if x > 200:
    print(x/0)
print(x)

The code can run just fine if that line never gets executed

There are many types of exceptions

In [ ]:
10.0**1000.0

In [ ]:
float('not a float')

In [ ]:
print(y)

You can even raise Exceptions your self

In [ ]:
raise Exception

In [ ]:
raise Exception('There was an error')

### Handling Errors

Try, Except blocks let you "catch" an Exception then decide what to do

In [ ]:
try:

    x = 10
    x / 0
    print('Nothing bad happened')

except Exception:

    print('Something bad happened')

print('We made it to the end')

An Exception was raised, but the code was able to keep running

This is good for handling things with variable inputs, like functions with crashing your code

In [ ]:
def divide(x, y):
    '''
    divides x by y

    Input:
    x: numerator (float)
    y: denominator (float)

    returns float
    '''

    try:
        res = x/y
    except ZeroDivisionError as e:
        print("Can't divide by 0")
        return None
    
    return res

In [ ]:
divide(5, 2)

In [ ]:
divide(5, 0)

In [ ]:
divide(5, '2')

Still raises an error for other Exception types
    
If we instead say Exception that would catch all Exception types

In [ ]:
def divide(x, y):
    '''
    divides x by y

    Input:
    x: numerator (float)
    y: denominator (float)

    returns float
    '''

    try:
        res = x/y
    except Exception as e:
        print(e)
        print("Can't divide by 0")
        return None
    
    return res

In [ ]:
divide(5, '2')

### Reading an error Traceback

The error message when an Exception is raised gives you the "Traceback", which tells you where in the code the error occurred 

In [ ]:
5 / 0

In [ ]:
def divide(x, y):
    '''
    divides x by y

    Input:
    x: numerator (float)
    y: denominator (float)

    returns float
    '''

    res = x/y
    
    return res

In [ ]:
divide(5, 0)

Traceback now points to 2 locations. 

Where you entered the function, 

then where in the function the error occured 

Now let's try looking at a traceback when there's an error using a function from a module, like pyplot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# make some data
a = np.arange(5)

b = np.arange(3)

# try to plot it
plt.plot(a, b)

Now there's several "layers" to this traceback, 

but the first entry is still where it started, where in your code you called that function

Then the last entry is where the exception is actually raised in the matplotlib code

The last line is still the error type and message, which should hopefully contain a clear message telling you what is wrong

    ValueError: x and y must have same first dimension, but have shapes (5,) and (3,)

Since we've gone through several functions, "x" and "y" may be disconnected from your actual input, making this sometimes harder to follow

though in this case it's easy enough to infer that x and y are your 2 arguments into plot()

The actual line in the matplotlib code where that Exception is raised can be seen here
https://github.com/matplotlib/matplotlib/blob/v3.10.x/lib/matplotlib/axes/_base.py#L494

Usually you really only need to care about the "layers" in your code and the error message

There could be a bug in matplotlib and other common packages, but more often than not it's in your code

### Finding bugs

When there is an Exception it's obvious something went wrong, but it's not always obvious where it started to go wrong

Let's make some code to take two lists of numbers
- multiply the lists element by element
- convert the resulting list to a numpy array
- multiply the whole array by 5  

In [ ]:
# function to multiply 2 numbers
def mult(a, b):
    
    return a*b


# our starting lists
list1 = [1, 2, '3', 4, 5]
list2 = [0, 2, 4, 6, 8]

# a list to populate 
list12 = []

# iterate through lists
for i in range(len(list1)):

    # get item from each list
    a = list1[i]
    b = list2[i]
    # multiply 
    res = mult(a, b)

    list12.append(res)

# convert list to array
arr12 = np.array(list12)

arr_final = 5*arr12

print('My final array is, ', arr_final)

There's an error! Something must have went wrong

Not very obvious from the error message though, "out kwarg"?, strings?

Let's start a debugging investigation to try to figure out what's happening 

First, let's see what's in the array we operated on where the Exception occured, arr12

In [ ]:
print(arr12)

Anything seem off about it?

But how did that happen? 

A bug happened earlier in the code that did not raise an Exception

We now need to step through the code and investigate the results of the opertations performed to see where things started to go wrong

An easy way to find out what's going on through the code is to add some strategic print statements, printing useful information on the current state of variables throughout the code

It may also help to have some "narrating" print statements saying what's going on to help point to where in the code you are 

In [ ]:
# a list to populate 
list12 = []

print('Entering for loop')

# iterate through lists
for i in range(len(list1)):

    # get item from each list
    a = list1[i]
    b = list2[i]
    print("in loop iteration i=",i)
    print("a =",a)
    print("b =",b)
    # multiply 
    res = mult(a, b)
    print("a*b = ", res)

    list12.append(res)

print("Exiting for loop")

print("List12 = ", list12)

# convert list to array
arr12 = np.array(list12)

print("Converted list to numpy array")
print("arr12 =", arr12)

arr_final = 5*arr12

print('My final array is, ', arr_final)

Going back 1 step, looks like only 3333 was a string

Going back into the loop, everything looks fine, besides at i=2

4*3 = 3333?

Let's get some more information. Let's not just print a and b, let's check their types

In [ ]:
# a list to populate 
list12 = []

print('Entering for loop')

# iterate through lists
for i in range(len(list1)):

    # get item from each list
    a = list1[i]
    b = list2[i]
    print("in loop iteration i=",i)

    print("type(a) =", type(a))
    print("type(b) =", type(b))

    print("a =",a)
    print("b =",b)
    # multiply 
    res = mult(a, b)
    print("a*b = ", res)

    list12.append(res)

print("Exiting for loop")

print("List12 = ", list12)

# convert list to array
arr12 = np.array(list12)

print("Converted list to numpy array")
print("arr12 =", arr12)

arr_final = 5*arr12

print('My final array is, ', arr_final)

Oh looks like we may have found our issue

The list, list1 at index 2 has a string in it

In [ ]:
print(list1)
print(list1[2])

Hard to identify at first

In [ ]:
print(3)
print('3')

Let's fix list1, and try again

In [ ]:
# function to multiply 2 numbers
def mult(a, b):
    
    return a*b


# our starting lists
list1 = [1, 2, 3, 4, 5]
list2 = [0, 2, 4, 6, 8]

# a list to populate 
list12 = []

# iterate through lists
for i in range(len(list1)):

    # get item from each list
    a = list1[i]
    b = list2[i]
    # multiply 
    res = mult(a, b)

    list12.append(res)

# convert list to array
arr12 = np.array(list12)

arr_final = 5*arr12

print('My final array is, ', arr_final)

Sometimes there's a bug but there's no Exception

In [ ]:
def calc_chi2(model, data, error):
    '''
    Calculates chi2 value
    '''

    chi2 = np.sum( (data - model)/(error)**2 )
    
    return chi2

In [ ]:
# get model expectation
model = np.ones(5)

# make some data
data = 0.9*np.ones(5)

# 10% error on data
error = 0.1*data

# calc the chi2
chi2 = calc_chi2(model, data, error)
print(chi2)

Is there something wrong with this chi2 value?

Function does one line for the computation, let's try doing multiple steps and checking it as we go

In [ ]:
def calc_chi2(model, data, error):
    '''
    Calculates chi2 value
    '''

    chi2i_num = (data - model)**2
    print('chi2i_num =')
    print(chi2i_num)

    chi2i_denom = error**2
    print('chi2i_denom =')
    print(chi2i_denom)

    chi2i = chi2i_num / chi2i_denom
    print('chi2i =')
    print(chi2i)

    chi2 = np.sum( chi2i )
    
    return chi2

In [ ]:
# calc the chi2
chi2 = calc_chi2(model, data, error)
print(chi2)

Why's it different this time? 

In [ ]:
# original 
def calc_chi2(model, data, error):
    '''
    Calculates chi2 value
    '''

    chi2 = np.sum( (data - model)/(error)**2 )
    
    return chi2

### Preventing Bugs

The best method to prevent Bugs is writing clear, easy to follow code

The next best is by testing each section of your code 

Tests can be made by giving an easy example that you already know the answer to 

When designing a test it should be simple enough that you know the answer, but should also be able to test most of what can go wrong

Let's make a test for our wrong chi2 function

In [ ]:
# original 
def calc_chi2_wrong(model, data, error):
    '''
    Calculates chi2 value
    '''

    chi2 = np.sum( (data - model)/(error)**2 )
    
    return chi2

In the function there is arithmatic and a sum. We should make sure both of those elements are tested. 

To test the arithmatic let's avoid 0s and 1s that give the same answer when sums and squares are done. 

We need to make up something for the 3 inputs

Let's do 

model = [2, 2],
data = [4, 4],
error = [0.5, 0.5]

We can work that out ourselves

( ((4 - 2) / (0.5)^2) + ((4 - 2) / (0.5)^2) )

( (2 / 0.5)^2 + (2 / 0.5)^2 )

( (4)^2 + (4)^2 )

( 16 + 16 )

32

In [ ]:
model = np.array([2, 2])
data = np.array([4, 4])
error = np.array([0.5, 0.5])

true_ans = 32

ans = calc_chi2_wrong(model, data, error)

print('our answer is, ',ans)

if ans == true_ans:
    print('Correct!')
else:
    print("Wrong :(")

In [ ]:
# Extra parentheses
def calc_chi2_fixed(model, data, error):
    '''
    Calculates chi2 value
    '''

    chi2 = np.sum( ( (data - model)/(error) )**2  )
    
    return chi2

In [ ]:
model = np.array([2, 2])
data = np.array([4, 4])
error = np.array([0.5, 0.5])

true_ans = 32

ans = calc_chi2_fixed(model, data, error)

print('our answer is, ',ans)

if ans == true_ans:
    print('Correct!')
else:
    print("Wrong :(")

Designing tests like these for your functions and major sections of code can help prevent bugs from happening and help prevent bugs that are hard to trace down